In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import keras
from keras.models import *
from keras.layers import *

from keras import layers
import keras.backend as K

Using TensorFlow backend.


In [2]:



def one_side_pad(x):
    x = ZeroPadding2D((1, 1))(x)
    x = Lambda(lambda x: x[:, :-1, :-1, :])(x)
    return x


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at
                     main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1),name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1),name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block,
               strides=(2, 2)):

    filters1, filters2, filters3 = filters

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides, name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, dilation_rate=(2, 2), padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1),name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides, name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x


def get_resnet50_encoder(input_height=256,  input_width=256, classes=1000):

    #assert input_height % 32 == 0
    #assert input_width % 32 == 0
 
    img_input = Input(shape=(input_height, input_width, 3))
    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7),strides=(2, 2), name='conv1')(x)
    
    #x = (concatenate([x1,x_re]))
    
    f1 = x

    x = BatchNormalization(name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3),strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
    f2 = one_side_pad(x)

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    f3 = x

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
    f4 = x

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    f5 = x

    x = AveragePooling2D((7, 7), name='avg_pool')(x)
    # f6 = x

    return img_input, [f1, f2, f3, f4, f5]



def _unet(n_classes, encoder, l1_skip_conn=True, input_height=416,
          input_width=608,encoder_level=4):

    img_input, levels = encoder(
        input_height=input_height, input_width=input_width)
    [f1, f2, f3, f4, f5] = levels
    o = f5

    o = (ZeroPadding2D((1, 1)))(o)
    o = (SeparableConv2D(512, (3, 3), padding='valid'))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2)))(o)
    o = (concatenate([o, f4]))
    o = (ZeroPadding2D((1, 1)))(o)
    o = (SeparableConv2D(256, (3, 3), padding='valid'))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2)))(o)
    o = (concatenate([o, f3], axis=-1))
    o = (ZeroPadding2D((1, 1)))(o)
    o = (SeparableConv2D(128, (3, 3), padding='valid'))(o)
    o = (BatchNormalization())(o)
    
    
    o = (UpSampling2D((2, 2)))(o)
    o = (concatenate([o, f2], axis=-1))
    o = (ZeroPadding2D((1, 1)))(o)
    o = (SeparableConv2D(128, (3, 3), padding='valid'))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2)))(o)

    if l1_skip_conn:
        o = (concatenate([o, f1], axis=-1))

    o = (ZeroPadding2D((1, 1)))(o)
    o = (SeparableConv2D(64, (3, 3), padding='valid'))(o)
    o = (BatchNormalization())(o)
    
    o = (UpSampling2D((2, 2)))(o)
    o = (ZeroPadding2D((1, 1)))(o)
    o = (SeparableConv2D(64, (3, 3), padding='valid'))(o)
    o = (BatchNormalization())(o)

    o = Conv2D(n_classes, (3, 3), padding='same')(o)

  
    o = Activation('softmax')(o)
    model = Model(img_input, o)

    return model





def resnet50_unet(n_classes, input_height=576, input_width=768,
                    encoder_level=3):
    
    model = _unet(n_classes, get_resnet50_encoder,
                  input_height=input_height, input_width=input_width)
    model.model_name = "resnet50_unet"
    model.save('path_to_my_model.h5')
    return model
model = resnet50_unet(3, input_height=None, input_width=None, encoder_level=4)
















Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead








In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [4]:
import glob
import re
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import numpy as np

def sorted_nicely( li ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(li, key = alphanum_key)
    
orig_list = glob.glob('/home/sami/Downloads/Canola Dataset/labeled/Final 98/Aug_xtrain/split/*.png')
orig_list = sorted_nicely(orig_list)
anot_list = glob.glob('/home/sami/Downloads/Canola Dataset/labeled/Final 98/Aug_ytrain/ysplit/*.png')
#anot_list = glob.glob('/home/sami/Downloads/Canola Dataset/labeled/Final 98/Aug_ytrain/ysplitnew/*.png')


anot_list = sorted_nicely(anot_list)
orig_train, orig_test, anot_train, anot_test = train_test_split(orig_list,anot_list,test_size = 0.1)

x_train = np.asarray(orig_train)
y_train = np.asarray(anot_list)
x_val   = np.asarray(orig_test)
y_val   = np.asarray(anot_test)

Y_train = np.array([image.img_to_array(image.load_img(fname)) for fname in y_train])
X_train = np.array([image.img_to_array(image.load_img(fname)) for fname in x_train]) #
Y_val = np.array([image.img_to_array(image.load_img(fname)) for fname in y_val])
X_val = np.array([image.img_to_array(image.load_img(fname)) for fname in x_val])
"""
x_train = np.asarray(orig_list)
y_train = np.asarray(anot_list)
Y_train = np.array([image.img_to_array(image.load_img(fname,target_size = (576,768))) for fname in y_train])

X_train = np.array([image.img_to_array(image.load_img(fname,target_size = (576,768))) for fname in x_train])"""

ValueError: Found input variables with inconsistent numbers of samples: [0, 6640]

In [ ]:
"""import glob
import re
from keras.preprocessing import image
from sklearn.model_selection import train_test_split


def sorted_nicely( li ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(li, key = alphanum_key)


orig_list = glob.glob('/home/sami/Downloads/Canola Dataset/labeled/Final 98/Aug_xtest/*.jpg')
orig_list = sorted_nicely(orig_list)
anot_list = glob.glob('/home/sami/Downloads/Canola Dataset/labeled/Final 98/Aug_ytest/*.png')
anot_list = sorted_nicely(anot_list)
#orig_train, orig_test, anot_train, anot_test = train_test_split(orig_list,anot_list,test_size = 0.1)

x_train = np.asarray(orig_list)
y_train = np.asarray(anot_list)
#x_val   = np.asarray(orig_test)
#y_val   = np.asarray(anot_test)
#,target_size = (576,768)
Y_train = np.array([image.img_to_array(image.load_img(fname)) for fname in y_train])
X_train = np.array([image.img_to_array(image.load_img(fname)) for fname in x_train])
#Y_val = np.array([image.img_to_array(image.load_img(fname,target_size = (576,768))) for fname in y_val])
#X_val = np.array([image.img_to_array(image.load_img(fname,target_size = (576,768))) for fname in x_val])"""
"""
x_train = np.asarray(orig_list)
y_train = np.asarray(anot_list)
Y_train = np.array([image.img_to_array(image.load_img(fname,target_size = (576,768))) for fname in y_train])

X_train = np.array([image.img_to_array(image.load_img(fname,target_size = (576,768))) for fname in x_train])"""